# Mem0 - Test Azure AI Search Vector DB

### Install Required Packages

In [2]:
!pip install --upgrade  azure-search-documents

#11.4.0b8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.7/297.7 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: azure-search-documents
    Found existing installation: azure-search-documents 11.4.0b8
    Uninstalling azure-search-documents-11.4.0b8:
      Successfully uninstalled azure-search-documents-11.4.0b8


# Configs

In [25]:
import os
from mem0 import Memory
import warnings 
warnings.filterwarnings('ignore')

config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": "gpt-4o-mini",
            "temperature": 0.1,
            "max_tokens": 2000,
             "azure_kwargs" : {
                  "azure_deployment" : "gpt-4o-mini",
                  "api_version" : "2024-02-15-preview",
                  "azure_endpoint" : "https://aoaiswedencentralmo.openai.azure.com/",
                  "api_key" : "key"
              }
        }
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": "text-embedding-ada-002",
            "azure_kwargs" : {
                  "api_version" : "2024-02-15-preview",
                  "azure_deployment" : "text-embedding-ada-002",
                  "azure_endpoint" : "https://aoaiswedencentralmo.openai.azure.com/",
                  "api_key": "key"
              }
        }
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": "ai-search-mo",
            "api_key": "key",
            "embedding_model_dims": 1536 ,
            "use_compression": True,
            "collection_name": "mem0_compression1"
        }
    }
}


### Test adding and searching

In [26]:
m = Memory.from_config(config)
m.add("Likes to play cricket and soccer on weekends", user_id="alice", metadata={"category": "hobbies"})



{'message': 'ok'}

In [27]:
related_memories = m.search(query="Help me plan my weekend.", user_id="alice")
print(related_memories)

[{'id': '135605dc-7a5a-4d5a-8c2d-82d4fdc7b3b7', 'memory': 'Likes to play cricket and soccer on weekends', 'hash': '285d07801ae42054732314853e9eadd7', 'metadata': {'category': 'hobbies'}, 'score': 0.8357886, 'created_at': '2024-10-28T13:26:51.138031-07:00', 'updated_at': '2024-10-28T13:51:18.592857-07:00', 'user_id': 'alice'}]


## Test get

In [28]:
memory_id = related_memories[0]['id']
memory = m.get(memory_id)
print(memory)

{'id': '135605dc-7a5a-4d5a-8c2d-82d4fdc7b3b7', 'memory': 'Likes to play cricket and soccer on weekends', 'hash': '285d07801ae42054732314853e9eadd7', 'metadata': {'category': 'hobbies'}, 'created_at': '2024-10-28T13:26:51.138031-07:00', 'updated_at': '2024-10-28T13:51:18.592857-07:00', 'user_id': 'alice'}


In [29]:
m.get_all(user_id="alice")

[{'id': '135605dc-7a5a-4d5a-8c2d-82d4fdc7b3b7',
  'memory': 'Likes to play cricket and soccer on weekends',
  'hash': '285d07801ae42054732314853e9eadd7',
  'metadata': {'category': 'hobbies'},
  'created_at': '2024-10-28T13:26:51.138031-07:00',
  'updated_at': '2024-10-28T13:51:18.592857-07:00',
  'user_id': 'alice'}]

In [35]:
messages = [{"role": "user", "content": "I recently tried chicken and I loved it. I'm thinking of trying more non-vegetarian dishes.."}]
m.add(messages, user_id="alex")

# Add second message to update history
messages.append({'role': 'user', 'content': 'I turned vegetarian now.'})
m.add(messages, user_id="alex")



{'message': 'ok'}

In [36]:
alex_mem= m.get_all(user_id="alex")
print(alex_mem)

[{'id': 'ca691fb9-64b0-44ad-90a0-c967f240c241', 'memory': 'Turned vegetarian now', 'hash': 'ce7710c57995e028a70c449df75be943', 'metadata': None, 'created_at': '2024-10-28T13:52:30.060397-07:00', 'updated_at': '2024-10-28T13:52:33.250155-07:00', 'user_id': 'alex'}]


In [37]:
memory_id = alex_mem[0]["id"]
history = m.history(memory_id)
print(history)

[{'id': 'a01015ad-c7c9-4eb6-aa74-b6e7cdb5970f', 'memory_id': 'ca691fb9-64b0-44ad-90a0-c967f240c241', 'old_memory': None, 'new_memory': 'Recently tried chicken and loved it', 'event': 'ADD', 'created_at': '2024-10-28T13:52:30.060397-07:00', 'updated_at': None}, {'id': 'd48d157c-a9a6-405a-aab0-5fbbc789a84e', 'memory_id': 'ca691fb9-64b0-44ad-90a0-c967f240c241', 'old_memory': 'Recently tried chicken and loved it', 'new_memory': 'Turned vegetarian now', 'event': 'UPDATE', 'created_at': '2024-10-28T13:52:30.060397-07:00', 'updated_at': '2024-10-28T13:52:33.250155-07:00'}]


## Test update and delete

In [38]:
message = "I recently tried chicken and I loved it. I'm thinking of trying more non-vegetarian dishes.."
m.update(memory_id, message)

{'message': 'Memory updated successfully!'}

In [39]:
alex_mem= m.get_all(user_id="alex")
print(alex_mem)

[{'id': 'ca691fb9-64b0-44ad-90a0-c967f240c241', 'memory': "I recently tried chicken and I loved it. I'm thinking of trying more non-vegetarian dishes..", 'hash': 'ce7710c57995e028a70c449df75be943', 'metadata': None, 'created_at': '2024-10-28T13:52:30.060397-07:00', 'updated_at': '2024-10-28T13:52:50.254822-07:00', 'user_id': 'alex'}]


In [40]:
m.delete(memory_id)

{'message': 'Memory deleted successfully!'}

In [41]:
alex_mem= m.get_all(user_id="alex")
print(alex_mem)

[]


In [42]:
alice_mem= m.get_all(user_id="alice")
print(alice_mem)

[{'id': '135605dc-7a5a-4d5a-8c2d-82d4fdc7b3b7', 'memory': 'Likes to play cricket and soccer on weekends', 'hash': '285d07801ae42054732314853e9eadd7', 'metadata': {'category': 'hobbies'}, 'created_at': '2024-10-28T13:26:51.138031-07:00', 'updated_at': '2024-10-28T13:51:18.592857-07:00', 'user_id': 'alice'}]


In [43]:
m.delete_all(user_id="alice")

{'message': 'Memories deleted successfully!'}

In [44]:
alice_mem= m.get_all(user_id="alice")
print(alice_mem)

[]


In [45]:
m.get_all(user_id="alex")

[]

In [46]:
m.reset()